<img src="images/header.png" alt="Logo UCLM-ESII" align="right">

<br><br><br><br>
<h2><font color="#92002A" size=4>Trabajo Fin de Grado</font></h2>

<h1><font color="#6B001F" size=5>Generación automática de playlist de canciones <br> mediante técnicas de minería de datos</font></h1>
<h2><font color="#92002A" size=3>Parte 6 - Generación del DataSet de pruebas</font></h2>

<br>
<div style="text-align: right">
    <font color="#B20033" size=3><strong>Autor</strong>: <em>Miguel Ángel Cantero Víllora</em></font><br>
    <br>
    <font color="#B20033" size=3><strong>Directores</strong>: <em>José Antonio Gámez Martín</em></font><br>
    <font color="#B20033" size=3><em>Juan Ángel Aledo Sánchez</em></font><br>
    <br>
<font color="#B20033" size=3>Grado en Ingeniería Informática</font><br>
<font color="#B20033" size=2>Escuela Superior de Ingeniería Informática | Universidad de Castilla-La Mancha</font>

</div>

---

<br>


<a id="indice"></a>
<h2><font color="#92002A" size=5>Índice</font></h2>

<br>

* [1. Introducción](#section1)
* [2. Lectura de datos](#section2)
* [3. Lectura del conjunto de pistas](#section3)
* [4. Obtención de playlists válidas](#section4)
* [5. Generación del conjunto de prueba](#section5)

<br>

---

In [1]:
# Permite establecer la anchura de la celda
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
import datetime
import json
import os
import pandas as pd
import random
import zipfile

from collections import defaultdict
from operator import itemgetter
from tqdm.notebook import tqdm as tqdm_nb

---

<br>


<a id="section1"></a>
## <font color="#92002A">1 - Introducción</font>
<br>

Para crear el conjunto de prueba, nos basaremos también en el conjunto que se ofreció como reto en el [*RecSys Challenge 2018*](https://recsys-challenge.spotify.com/).

Dicho conjunto se utilizaba para evaluar el sistema de recomendación creado. Los participantes del reto tenían el conjunto de forma incompleta y, una vez creado el sistema, enviaban los resultados a la organización para devolverles la puntuación obtenida.

En nuestro caso, vamos a crear dos ficheros: uno que contendrá las playlists con su contenido completo y otro con las playlists incompletas (con la posibilidad de que el nombre de la playlist no figure).

<br>

---

<br>

<a id="section11"></a>
### <font color="#92002A">1.1 - Formato</font>
<br>

El conjunto consta de un único diccionario almacenado en un fichero *JSON*, con los siguientes campos:

* ***date***: Fecha en la que se ha generado (formato *timestamp*).
* ***version***: Versión del conjunto de datos.
* ***playlists***: Array de 10.000 playlist incompletas. Cada elemento de la lista es un diccionario que posee los siguientes campos:

    * ***pid***: [integer] Identificador de la playlist.
    * ***name***: [string] Título de la lista. (Opcional)
    * ***num_holdouts***: [string] Número de pistas que han sido omitidas de la lista.
    * ***tracks***: Lista de diccionarios con información sobre las pistas (la lista puede estar vacía). Cada diccionario contiene los siguientes campos:
        * *pos*: Posición de la pista dentro de la playlist.
        * *track_name*: Título de la pista.
        * *track_uri*: Dirección Spotify de la pista.
        * *album_name*: Título del álbum.
        * *album_uri*: Dirección Spotify del álbum.
        * *artist_name*: Nombre del artista.
        * *artist_uri*: Dirección Spotify del artista.
        * *duration_ms*: Duración en milisegundos de la pista.
    * ***num_samples***: [string] Número de pistas que contiene la lista.
    * ***num_tracks***: [string] Número *total* de pistas que contiene la lista.

<br>

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i> __Nota 1__:
        \begin{equation*}
        len(tracks) == num\_samples
        \end{equation*}
</div>

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i> __Nota 2__:
        \begin{equation*}
        num\_samples + num\_holdouts == num\_tracks
        \end{equation*}
</div>

<br>

---

<br>

<a id="section12"></a>
### <font color="#92002A">1.2 - Categorías del conjunto de prueba</font>
<br>

Las 10.000 playlists están agrupadas en 10 categorías de desafíos diferentes, con 1.000 listas en cada categoría:

1. Predecir las pistas para una playlist dado sólo su título.
2. Predecir las pistas para una playlist dado su título y la primera pista.
3. Predecir las pistas para una playlist dado su título y las primeras 5 pistas.
4. Predecir las pistas para una playlist dadas las primeras 5 pistas (sin título).
5. Predecir las pistas para una playlist dado su título y las primeras 10 pistas.
6. Predecir las pistas para una playlist dadas las primeras 10 pistas (sin título).
7. Predecir las pistas para una playlist dado su título y las primeras 25 pistas.
8. Predecir las pistas para una playlist dado su título y 25 pistas aleatorias.
9. Predecir las pistas para una playlist dado su título y las primeras 100 pistas.
10. Predecir las pistas para una playlist dado su título y 100 pistas aleatorias.

---

<br>

<a id="section13"></a>
### <font color="#92002A">1.3 - Restricciones</font>
<br>

Las playlists del conjunto de prueba cumplen con lo siguiente:
- Todas las pistas del conjunto de prueba están en el conjunto final (*MPD*).
- Todas las pistas por predecir (*holdout*) están en el conjunto final.

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<a id="section2"></a>
## <font color="#92002A">2 - Lectura de datos</font>

<br>

Al igual que hicimos para generar el conjunto final, vamos a leer el fichero donde hemos almacenado la información relativa a las playlists y vamos a cargar desde la playlist que figura en la posición *1.000.000* en adelante (que son las que tenemos para crear el conjunto de prueba).

<br>

In [3]:
# Variables globales

# Directorios donde almacenaremos el DataSet que vamos a generar
MPD_PATH = 'MPD'
MPD_TEST_PATH = 'MPD_TEST'

# Directorio donde se encuentran almacenadas las playlists que 
# hemos descargado
PLS_SETS_PATH = 'pls_sets'

# Directorio empleado para guardar/leer aquellos datos generados
# que resultan de interés para diferentes tareas
DATA_PATH = 'data'

In [4]:
# Ruta del fichero donde están contenidas las playlists
mpd_info_file = os.path.join(DATA_PATH, 'mpd_info_set.csv')

if os.path.isfile(mpd_info_file):
    # Cargamos a partir de la playlist desde donde comienza el conjunto de prueba
    df_mpd_test_info = pd.read_csv(mpd_info_file, sep=';', encoding='utf-8')[1000000:]
    print(f"Playlists obtenidas: {len(df_mpd_test_info)}")
else:
    print("¡ERROR!: Fichero 'mpd_info_set.csv' no encontrado.")

Playlists obtenidas: 19338


<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<a id="section3"></a>
## <font color="#92002A">3 - Lectura del conjunto de pistas</font>

<br>


Para facilitar el proceso de lectura de las listas de canciones de las playlists, vamos a crear un diccionario en el cual estableceremos como clave el nombre del fichero y cuyo valor será una lista con los identificadores de las playlists que deseamos obtener su información:

<br>

In [5]:
pls_location_dict = defaultdict(list)

for _, row in df_mpd_test_info.iterrows():
    pls_location_dict[row['file_name']].append(row['id'])

<br>

Tras crear el diccionario, procedemos a obtener las listas de pistas que nos interesa guardar:

In [6]:
def get_test_pls_data(location_dict, pls_sets_path):
    pls_dict = dict()
    
    pbar = tqdm_nb(total=len(location_dict))
    for file_name, pls_ids in location_dict.items():
        zip_file_path = os.path.join(pls_sets_path, file_name)
        with zipfile.ZipFile(zip_file_path,'r') as zip_file:
            with zip_file.open(zip_file.namelist()[0]) as json_file:
                data = json_file.read()
                pls = json.loads(data.decode())
        for pl in pls:
            if pl['id'] in pls_ids:
                pls_dict[pl['id']] = pl
        pbar.update(1)
    pbar.close()
    
    return pls_dict

In [7]:
test_pls_dict = get_test_pls_data(pls_location_dict, PLS_SETS_PATH)
print(f'Obtenida la información de {len(test_pls_dict)} playlists')


Obtenida la información de 19338 playlists


<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<a id="section4"></a>
## <font color="#92002A">4 - Obtención de playlists válidas</font>

<br>

Antes de comenzar, vamos a obtener las pistas que hay en las playlists que conforman el conjunto de datos que hemos creado. Este paso lo realizamos para que en nuestro conjunto de prueba no tengamos playlists que contienen pistas desconocidas (no figuran en el conjunto de datos final).

<br>

In [8]:
#https://www.geeksforgeeks.org/python-get-values-of-particular-key-in-list-of-dictionaries/
def get_tracks_ids(mpd_path):
    tracks_set = set()
    
    for file in tqdm_nb(os.listdir(mpd_path)):
        zip_file_path = os.path.join(mpd_path, file)
        with zipfile.ZipFile(zip_file_path,'r') as zip_file:
            with zip_file.open(zip_file.namelist()[0]) as json_file:
                data = json_file.read()
                slice_pls = json.loads(data.decode())['playlists']
        for pl in slice_pls:
            tracks_set.update(list(map(itemgetter('track_uri'), pl['tracks'])))
    
    return tracks_set

In [9]:
tracks_set = get_tracks_ids(MPD_PATH)

Obtenemos los identificadores de las playlist que cumplen con el criterio establecido:

In [10]:
valid_pls = []

for p_id, p_data in test_pls_dict.items():
    tracks = list(map(itemgetter('track_uri'), p_data['tracks']))
    if all(t in tracks_set for t in tracks):
        valid_pls.append(p_id)
        
print(f'Número de playlists válidas : {len(valid_pls)}')

Número de playlists válidas : 11701


<br>

Por último, eliminamos del *DataFrame* aquellas listas que no nos son válidas:

In [11]:
df_mpd_test_info.set_index('id', inplace=True)
df_mpd_test_info = df_mpd_test_info.loc[valid_pls]

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<a id="section5"></a>
## <font color="#92002A">5 - Generación del conjunto de prueba</font>

<br>

Una vez obtenidas las playlist que podemos emplear en el conjunto de prueba. Procedemos a elegir qué listas pueden formar parte de una de las categorías en las que se ha dividido el conjunto. Lo único que debemos tener en cuenta es que las playlists tengan más pistas que el número de elementos que se ofrecen en cada categoría.

Por ejemplo: Para la categoría "*Predecir las pistas para una playlist dado su título y 100 pistas aleatorias*", tenemos que asegurarnos de que las listas que vamos a elegir tengan más de 100 pistas.

<br>

In [12]:
df_mpd_test_len = df_mpd_test_info[['num_tracks']].copy()

In [13]:
test_pls_id_list = []

# Categorías 9 y 10
category_pls = df_mpd_test_len[df_mpd_test_len.num_tracks > 100].sample(2000).index.to_list()
test_pls_id_list += category_pls
df_mpd_test_len.drop(category_pls, inplace=True) # Eliminamos las playlist que hemos seleccionado

# Categorías 7 y 8
category_pls = df_mpd_test_len[df_mpd_test_len.num_tracks > 25].sample(2000).index.to_list()
test_pls_id_list += category_pls
df_mpd_test_len.drop(category_pls, inplace=True) # Eliminamos las playlist que hemos seleccionado

# Categorías 5 y 6
category_pls = df_mpd_test_len[df_mpd_test_len.num_tracks > 10].sample(2000).index.to_list()
test_pls_id_list += category_pls
df_mpd_test_len.drop(category_pls, inplace=True) # Eliminamos las playlist que hemos seleccionado

# Categorías 3 y 4
category_pls = df_mpd_test_len[df_mpd_test_len.num_tracks > 5].sample(2000).index.to_list()
test_pls_id_list += category_pls
df_mpd_test_len.drop(category_pls, inplace=True) # Eliminamos las playlist que hemos seleccionado

# Categorías 1 y 2
category_pls = df_mpd_test_len.sample(2000).index.to_list() # Nos vale cualquiera de las restantes playlists
test_pls_id_list += category_pls
df_mpd_test_len.drop(category_pls, inplace=True) # Eliminamos las playlist que hemos seleccionado

test_pls_id_list.reverse()

<br>

Puesto que ya tenemos hemos establecido qué playlists conforman cada categoría, vamos a leer los datos de cada una e incorporarlos a una lista. Con este proceso crearemos el conjunto de prueba completo (figuran todos los nombres y pistas de las playlists).

<br>

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i> __Nota__: Los campos `num_holdouts` y `num_samples` los añadiremos durante la creación del conjunto de playlists incompletas.
</div>

In [14]:
test_set_list = []
current_pid = 1000000

for pl_id in test_pls_id_list:
    pl_dict = dict()
    pl_dict['pid'] = current_pid
    pl_dict['name'] = df_mpd_test_info.loc[pl_id]['name']
    pl_dict['num_holdouts'] = None
    pl_dict['num_samples'] = None
    pl_dict['num_tracks'] = int(df_mpd_test_info.loc[pl_id]['num_tracks'])
    pl_dict['tracks'] = test_pls_dict[pl_id]['tracks'][:]
    # No necesitamos el campo "added_at", lo borramos
    for pl_track in pl_dict['tracks']:
        del pl_track['added_at']
    
    test_set_list.append(pl_dict)
    
    current_pid += 1

<br>

Con el conjunto completo de playlists que conforman el conjunto de prueba, creamos el conjunto que contiene las playlists incompletas. 

Lo primero que vamos a hacer es crear una función para facilitar el proceso de eliminación de pistas según la configuración indicada:

In [15]:
# Función mediante la cual devolvemos la playlist incompleta que hemos indicado
# con la configuración que se ha establecido.
def remove_playlist_content(pl_complete, num_samples, random_tracks= False, remove_title = False):
    pl_incomplete = pl_complete.copy()
    if remove_title:
        pl_incomplete['name'] = ""
        
    if random_tracks:
        # Obtenemos n índices de pistas aleatorias
        random_ids = random.sample(range(int(pl_complete['num_tracks'])), num_samples)
        # Ordenamos los ids
        random_ids.sort()
        # Obtenemos las pistas y las añadimos a pl_incomplete['tracks']
        pl_incomplete['tracks'] = []
        for i in random_ids: 
            pl_incomplete['tracks'].append(pl_complete['tracks'][i])        
    else:
        pl_incomplete['tracks'] = pl_incomplete['tracks'][0:num_samples]
        
    pl_incomplete['num_holdouts'] = int(pl_complete['num_tracks']) - num_samples
    pl_incomplete['num_samples'] = num_samples
    
    return pl_incomplete

<br>

Por último, por cada categoría añadimos el conjunto de playlist con la configuración que hemos indicado y vamos incorporando las playlists a la lista `test_set_incomplete_list`:

In [16]:
test_set_incomplete_list = []

# 1 - Playlist dado sólo su título.
for pl in test_set_list[0:1000]:
    num_samples = 0
    pl_incomplete = remove_playlist_content(pl, num_samples)
    pl['num_samples'] = num_samples
    pl['num_holdouts'] = str(int(pl['num_tracks']) - num_samples)
    test_set_incomplete_list.append(pl_incomplete)
    
# 2 - Playlist dado su título y la primera pista.
for pl in test_set_list[1000:2000]:
    num_samples = 1
    pl_incomplete = remove_playlist_content(pl, num_samples)
    pl['num_samples'] = num_samples
    pl['num_holdouts'] = str(int(pl['num_tracks']) - num_samples)
    test_set_incomplete_list.append(pl_incomplete)
    
# 3 -  Playlist dado su título y las primeras 5 pistas.
for pl in test_set_list[2000:3000]:
    num_samples = 5
    pl_incomplete = remove_playlist_content(pl, num_samples)
    pl['num_samples'] = num_samples
    pl['num_holdouts'] = int(pl['num_tracks']) - num_samples
    test_set_incomplete_list.append(pl_incomplete)
    
# 4 - Playlist dadas las primeras 5 pistas (sin título).
for pl in test_set_list[3000:4000]:
    num_samples = 5
    pl_incomplete = remove_playlist_content(pl, num_samples, remove_title=True)
    pl['num_samples'] = num_samples
    pl['num_holdouts'] = int(pl['num_tracks']) - num_samples
    test_set_incomplete_list.append(pl_incomplete)
    
# 5 - Playlist dado su título y las primeras 10 pistas.
for pl in test_set_list[4000:5000]:
    num_samples = 10
    pl_incomplete = remove_playlist_content(pl, num_samples)
    pl['num_samples'] = num_samples
    pl['num_holdouts'] = int(pl['num_tracks']) - num_samples
    test_set_incomplete_list.append(pl_incomplete)
    
# 6 - Playlist dadas las primeras 10 pistas (sin título).
for pl in test_set_list[5000:6000]:
    num_samples = 10
    pl_incomplete = remove_playlist_content(pl, num_samples, remove_title=True)
    pl['num_samples'] = num_samples
    pl['num_holdouts'] = int(pl['num_tracks']) - num_samples
    test_set_incomplete_list.append(pl_incomplete)
    
# 7 - Playlist dado su título y las primeras 25 pistas.
for pl in test_set_list[6000:7000]:
    num_samples = 25
    pl_incomplete = remove_playlist_content(pl, num_samples)
    pl['num_samples'] = num_samples
    pl['num_holdouts'] = int(pl['num_tracks']) - num_samples
    test_set_incomplete_list.append(pl_incomplete)
    
# 8 - Playlist dado su título y 25 pistas aleatorias.
for pl in test_set_list[7000:8000]:
    num_samples = 25
    pl_incomplete = remove_playlist_content(pl, num_samples, random_tracks=True)
    pl['num_samples'] = num_samples
    pl['num_holdouts'] = int(pl['num_tracks']) - num_samples
    test_set_incomplete_list.append(pl_incomplete)
    
# 9 - Playlist dado su título y las primeras 100 pistas.
for pl in test_set_list[8000:9000]:
    num_samples = 100
    pl_incomplete = remove_playlist_content(pl, num_samples)
    pl['num_samples'] = num_samples
    pl['num_holdouts'] = int(pl['num_tracks']) - num_samples
    test_set_incomplete_list.append(pl_incomplete)
    
# 10 - Playlist dado su título y 100 pistas aleatorias.
for pl in test_set_list[9000:10000]:
    num_samples = 100
    pl_incomplete = remove_playlist_content(pl, num_samples, random_tracks=True)
    pl['num_samples'] = num_samples
    pl['num_holdouts'] = int(pl['num_tracks']) - num_samples
    test_set_incomplete_list.append(pl_incomplete)

<br>

Ahora volcamos el contenido de ambas listas (completa e incompleta) a su fichero *JSON* correspondiente y lo guardamos:

In [17]:
# Creamos la carpeta donde guardaremos los ficheros que pertenecen al
# conjunto de prueba
if not os.path.exists(MPD_TEST_PATH):
    os.makedirs(MPD_TEST_PATH)
    
# Creamos un diccionario para cada conjunto
test_set_complete = {'date' : str(datetime.datetime.now()),
                     'version': 'v1.1',
                     'playlists' : test_set_list
                    }

test_set = {'date' : str(datetime.datetime.now()),
            'version': 'v1.1',
            'playlists' : test_set_incomplete_list
           }

In [18]:
# Guardamos el fichero del conjunto de playlist de prueba incompleto

# Creamos el fichero .json
file_name = "mpd.test.json"
file_path = os.path.join(MPD_TEST_PATH, file_name)
with open(file_path, 'w') as f:
    json.dump(test_set,f,indent=4)
    
# Comprimimos el fichero que hemos creado
zip_file_path = file_path.replace('.json', '.zip')
with zipfile.ZipFile(zip_file_path,'w') as zip_file: 
     zip_file.write(file_path, compress_type=zipfile.ZIP_DEFLATED, arcname=file_name)

# Eliminamos el fichero creado para quedarnos con
# la versión comprimida
os.remove(file_path)

In [19]:
# Guardamos el fichero del conjunto de playlist de prueba incompleto

# Creamos el fichero .json
file_name = "mpd.test_complete.json"
file_path = os.path.join(MPD_TEST_PATH, file_name)
with open(file_path, 'w') as f:
    json.dump(test_set_complete,f,indent=4)
    
# Comprimimos el fichero que hemos creado
zip_file_path = file_path.replace('.json', '.zip')
with zipfile.ZipFile(zip_file_path,'w') as zip_file: 
     zip_file.write(file_path, compress_type=zipfile.ZIP_DEFLATED, arcname=file_name)

# Eliminamos el fichero creado para quedarnos con
# la versión comprimida
os.remove(file_path)

<br>

Para finalizar, vamos a eliminar del directorio `DATA_PATH` aquellos ficheros que ya no nos resultarán de utilidad:

In [20]:
os.remove(os.path.join(DATA_PATH, 'mpd_info_set.csv'))

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<div style="text-align: right"> <font size=6><i class="fa fa-graduation-cap" aria-hidden="true" style="color:#92002A"></i> </font></div>